**Table of contents**<a id='toc0_'></a>    
- [Robot Arm collision check](#toc1_)    
- [Starting pybullet](#toc2_)    
- [Initial Setup](#toc3_)    
- [Function Definitions](#toc4_)    
- [Running the Simulation](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Robot Arm collision check](#toc0_)

In this notebook, we will introduce how to perform collision detection between a "2-axis robot arm" and an "object".

(For a manual summarizing the functions available in pybullet, refer to [this link](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf).)

# <a id='toc2_'></a>[Starting pybullet](#toc0_)

In [1]:
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


# <a id='toc3_'></a>[Initial Setup](#toc0_)
We will perform initial setup such as creating the floor, generating box objects, creating the robot, and setting the camera position.

In [2]:
pybullet.resetSimulation() # Reset the simulation space
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # Add paths to necessary data for pybullet
pybullet.setGravity(0.0, 0.0, -9.8) # Set gravity as on Earth
time_step = 1./240.
pybullet.setTimeStep(time_step) # Set the time elapsed per step

# Load the floor
plane_id = pybullet.loadURDF("plane.urdf")

# Set the camera position and other parameters in GUI mode
camera_distance = 2.0
camera_yaw = 0.0 # deg
camera_pitch = -20 # deg
camera_target_position = [0.0, 0.0, 0.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

# Load the robot
arm_start_pos = [0, 0, 0.1]  # Set the initial position (x, y, z)
arm_start_orientation = pybullet.getQuaternionFromEuler([0, 0, 0])  # Set the initial orientation (roll, pitch, yaw)
arm_id = pybullet.loadURDF("../urdf/simple2d_arm.urdf", arm_start_pos, arm_start_orientation, useFixedBase=True) # Fix the root link with useFixedBase=True to prevent the robot from falling

# Set the camera position and other parameters in GUI mode
camera_distance = 1.5
camera_yaw = 180.0 # deg
camera_pitch = -10 # deg
camera_target_position = [0.0, 0.0, 1.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

# Load the box
## Determine the weight, size, position, and orientation of the box
mass = 5 # kg
box_size = [0.3, 0.6, 0.5]
position = [0.8, 0, 0.5]
orientation = [1, 0, 0, 0] # Quaternion
box_collision_id = pybullet.createCollisionShape(pybullet.GEOM_BOX, halfExtents=box_size)
box_visual_id = pybullet.createVisualShape(pybullet.GEOM_BOX, halfExtents=box_size, rgbaColor=[1,0,0,1]) # Red, semi-transparent
box_body_id = pybullet.createMultiBody(mass, box_collision_id, box_visual_id, position, orientation)

ven = Mesa
ven = Mesa
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: target_position_vertual_link


# <a id='toc4_'></a>[Function Definitions](#toc0_)
We will define the following functions necessary for enabling/disabling collisions between objects.
- `GetNumLinks`: Get the number of links of the specified object
- `EnableCollisionBetweenObjects`: Enable/disable collision between two objects
- `JudgeContact`: Determine whether two objects are in contact

In [3]:
def get_num_links(objId):
    """
    Get the number of links of the specified object.
    Parameters
    ----------
    objId : int
        Index of the object.
        
    Returns
    -------
    linkNum : int
        Number of links of the object.
    """
    current_link_idx = 0
    link_num = 0
    while True:
        result = pybullet.getLinkState(objId, current_link_idx)
        if result is None:
            link_num = current_link_idx + 1
            break
        current_link_idx += 1
    return link_num

def enable_collision_between_objects(obj1_id, obj2_id, enable):
    """
    Enable/disable collision between the specified objects.
    Parameters
    ----------
    obj1_id : int
        Index of the first object.
    obj2_id : int
        Index of the second object.
    enable : bool
        True to enable collision detection, False to disable it.

    Returns
    -------
    None
    """
    # Get the number of links of each object
    obj1_link_num = get_num_links(obj1_id)
    obj2_link_num = get_num_links(obj2_id)

    # Set collision detection to enable/disable
    for obj1_link_idx in range(-1, obj1_link_num):
        for obj2_link_idx in range(-1, obj2_link_num):
            pybullet.setCollisionFilterPair(obj1_id, obj2_id, obj1_link_idx, obj2_link_idx, enable)

def judge_collision(obj1_id, obj2_id):
    """
    Determine whether two objects are in contact.
    Parameters
    ----------
    obj1_id : int
        Index of the first object.
    obj2_id : int
        Index of the second object.

    Returns
    -------
    isCollision : bool
        True if the two objects are in contact, False otherwise.
    """
    # Get contact information between the two objects
    pts = pybullet.getClosestPoints(obj1_id, obj2_id, distance=100)

    # Determine whether the two objects are in contact
    is_collision = False
    for pt in pts:
        distance = pt[8]
        # Determine whether there are points in contact between the two objects (i.e., points with a distance less than 0)
        if distance < 0.0:
            is_collision = True
            break
    
    return is_collision

# <a id='toc5_'></a>[Running the Simulation](#toc0_)
Move the robot arm and perform collision detection with the object.

When you run the code below, you can control the robot arm with the mouse in the GUI screen. If the robot arm touches the box, "collision" will be displayed on the screen.

In [4]:
import time

pybullet.setRealTimeSimulation(1) # Enable real-time simulation (allows objects to be manipulated with the mouse in the GUI screen)

# Disable collision detection between the robot arm and the box
enable_collision_between_objects(arm_id, box_body_id, False)

# Display whether the robot arm and the box are colliding on the screen
while (pybullet.isConnected()):

    # Determine whether the robot arm and the box are colliding
    is_collision = judge_collision(arm_id, box_body_id)

    # If they are colliding, display it on the screen
    if is_collision:
        pybullet.addUserDebugText("collision", [0.5, 0.0, 1.5], textSize=5, lifeTime=0.1, textColorRGB=[1,0,0])
    
    time.sleep(time_step)